<h1><center>Trabajo Práctico 2: Entrenamiento y evaluación de modelos</center></h1>

## 1. Métrica ##

La métrica que vamo a utilizar es **Precision**

#### Por qué Precision?
Nuestro interes principal es no equivocarnos al marcar como interesados a los clientes, ya que para el negocio sería fundamental no ofrecer tarjetas a personas no interesadas.

La finalidad de Precision consiste en encontrar todos los clientes que estemos seguros que van a estar interesados, de la cantidad total que se marca como tal. Si el porcentaje de acierto resultante es bajo, estaríamos marcando como interesados a personas que no lo son.

## Librerías ##


In [ ]:
%matplotlib inline
import warnings
import numpy as np
import pandas as pd 
import matplotlib
import keras
import h5py
import PIL
import seaborn as sns
import sklearn
import pytz
import plotly.graph_objects as go

from matplotlib import pyplot as plt

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelBinarizer, QuantileTransformer, MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn import datasets
import plotly.express as px 

from sklearn import metrics
from collections import defaultdict

from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

plt.rcParams.update({
    "font.family": ["serif"],
    "font.sans-serif": ["Roboto"],
    "font.size": 9,
    "axes.labelsize": 11,
    "axes.titlesize": 13,
    "xtick.labelsize": 11,
    "ytick.labelsize": 11,
    "legend.fontsize": 11,
    'figure.figsize': (15.0, 4.0),
    'axes.grid': False,
    'axes.spines.left': True,
    'axes.spines.right': True,
    'axes.spines.top': True,
    'axes.spines.bottom': True,
})

np.set_printoptions(suppress=True)
warnings.filterwarnings('ignore')

## Train ##


In [ ]:
data_TC = pd.read_csv('train.csv')

BETTER_COLUMN_NAMES = {
    'ID': 'id',
    'Gender': 'sexo',
    'Age': 'edad',
    'Region_Code': 'codigo_region',
    'Occupation': 'ocupacion',
    'Channel_Code': 'codigo_canal',
    'Vintage': 'antiguedad',
    'Credit_Product': 'tiene_producto_credito_activo',
    'Avg_Account_Balance': 'saldo_promedio_cuenta',
    'Is_Active': 'es_activo',
    'Is_Lead': 'esta_interesado',
}
data_TC.rename(columns=BETTER_COLUMN_NAMES, inplace=True)

data_TC.set_index('id', inplace=True)

DATA_MODIFICADA = data_TC.copy(deep=True)

In [ ]:
DATA_ORIGINAL = data_TC.copy(deep=True)

## 2. Técnica feature engineering ##

#### **Quantile Transformation (Edades)** 
Utilizaremos esta técnica para llevar los datos a una distribución uniforme o normal, generando robustez para los outliers.

Usamos 10 quantiles.

In [ ]:
number_of_quantile = 10     
DATA_MODIFICADA['Quantiles_edad'],edges = pd.qcut(x=DATA_MODIFICADA['edad'], q=number_of_quantile, labels=False, retbins=True )
print('values of edges: ' + str(edges))

#Evitar Nans:
ed = np.delete(edges, 0)
ed = np.delete(ed, -1)
ed = np.append(np.append([DATA_MODIFICADA.edad.min()], [ed]), [DATA_MODIFICADA.edad.max()])
print('\nvalues of ed: ' + str(ed))

# Transform the array as an IntervalIndex
Interval_Index = pd.IntervalIndex.from_breaks(ed,closed='right',dtype='interval[int64]')
print('\nvalues of Interval_Index: ' + str(Interval_Index))

# Create a column displaying the interval
DATA_MODIFICADA['quantile_interval'] = pd.cut(DATA_MODIFICADA['edad'], bins=Interval_Index)

dict_inter_quantile = pd.Series(DATA_MODIFICADA['quantile_interval'].unique().sort_values(ascending=False), name='interval').reset_index()
dict_inter_quantile.columns = ['Quantiles_edad', 'quantile_interval']
dict_inter_quantile = dict_inter_quantile.set_index('quantile_interval')

#DATA_MODIFICADA.sample(8)

In [ ]:
df = px.data.tips()

fig = px.histogram(DATA_MODIFICADA, x="edad",color="esta_interesado", marginal = 'box',title='Distribución sin quantiles')
fig.update_layout(bargap=0.2)
fig.show()

fig = px.histogram(DATA_MODIFICADA,x="Quantiles_edad",color="esta_interesado", marginal = 'box',title='Distribución por quantiles')
fig.update_layout(bargap=0.2)
fig.show()

#### **Quantile Transformation (Saldo promedio de cuenta)** 
Utilizaremos esta técnica para llevar los datos a una distribución uniforme o normal, generando robustez para los outliers.

Usamos 20 quantiles.

In [ ]:
number_of_quantile = 20
DATA_MODIFICADA['Quantiles_saldos'],edges = pd.qcut(x=DATA_MODIFICADA['saldo_promedio_cuenta'], q=number_of_quantile, labels=False, retbins=True )
print('values of edges: ' + str(edges))

#Evitar Nans:
ed = np.delete(edges, 0)
ed = np.delete(ed, -1)
ed = np.append(np.append([DATA_MODIFICADA.saldo_promedio_cuenta.min()], [ed]), [DATA_MODIFICADA.saldo_promedio_cuenta.max()])
print('\nvalues of ed: ' + str(ed))

# Transform the array as an IntervalIndex
Interval_Index = pd.IntervalIndex.from_breaks(ed,closed='right',dtype='interval[int64]')
print('\nvalues of Interval_Index: ' + str(Interval_Index))

# Create a column displaying the interval
DATA_MODIFICADA['Intervalos saldos'] = pd.cut(DATA_MODIFICADA['saldo_promedio_cuenta'], bins=Interval_Index)

dict_inter_quantile = pd.Series(DATA_MODIFICADA['Intervalos saldos'].unique().sort_values(ascending=False), name='interval').reset_index()
dict_inter_quantile.columns = ['Quantiles_saldos', 'Intervalos saldos']
dict_inter_quantile = dict_inter_quantile.set_index('Intervalos saldos')

#DATA_MODIFICADA.sample(8)

In [ ]:
df = px.data.tips()

fig = px.histogram(DATA_MODIFICADA, x="saldo_promedio_cuenta",color="esta_interesado", marginal = 'box',title='Distribución sin quantiles')
fig.update_layout(bargap=0.2)
fig.show()

fig = px.histogram(DATA_MODIFICADA,x="Quantiles_saldos",color="esta_interesado", marginal = 'box', title='Distribución por quantiles')
fig.update_layout(bargap=0.2)
fig.show()

#### **Quantile Transformation (antiguedad)** 
Utilizaremos esta técnica para llevar los datos a una distribución uniforme o normal, generando robustez para los outliers.

Usamos 5 quantiles.

In [ ]:
number_of_quantile = 5
DATA_MODIFICADA['Quantiles_antiguedad'],edges = pd.qcut(x=DATA_MODIFICADA['antiguedad'], q=number_of_quantile, labels=False, retbins=True )
print('values of edges: ' + str(edges))

#Evitar Nans:
ed = np.delete(edges, 0)
ed = np.delete(ed, -1)
ed = np.append(np.append([DATA_MODIFICADA.antiguedad.min()], [ed]), [DATA_MODIFICADA.antiguedad.max()])
print('\nvalues of ed: ' + str(ed))

# Transform the array as an IntervalIndex
Interval_Index = pd.IntervalIndex.from_breaks(ed,closed='right',dtype='interval[int64]')
print('\nvalues of Interval_Index: ' + str(Interval_Index))

# Create a column displaying the interval
DATA_MODIFICADA['Intervalos antiguedad'] = pd.cut(DATA_MODIFICADA['antiguedad'], bins=Interval_Index)

dict_inter_quantile = pd.Series(DATA_MODIFICADA['Intervalos antiguedad'].unique().sort_values(ascending=False), name='interval').reset_index()
dict_inter_quantile.columns = ['Quantiles_antiguedad', 'Intervalos antiguedad']
dict_inter_quantile = dict_inter_quantile.set_index('Intervalos antiguedad')

#DATA_MODIFICADA.sample(8)

In [ ]:
df = px.data.tips()

fig = px.histogram(DATA_MODIFICADA, x="antiguedad",color="esta_interesado", marginal = 'box',title='Distribución sin quantiles')
fig.update_layout(bargap=0.2)
fig.show()

fig = px.histogram(DATA_MODIFICADA,x="Quantiles_antiguedad",color="esta_interesado", marginal = 'box',
                   title='Distribución por quantiles')
fig.update_layout(bargap=0.2)
fig.show()

## Preparación Datos ##


Eliminamos la columnas que no vamos a utilizar luego de la creación de los Quantiles.

In [ ]:
DATA_MODIFICADA.shape 

In [ ]:
DATA_MODIFICADA = DATA_MODIFICADA.drop(['edad', 'antiguedad', 'saldo_promedio_cuenta', 'quantile_interval','Intervalos saldos','Intervalos antiguedad'], axis=1)

In [ ]:
DATA_MODIFICADA.shape 

## 3. Qué modelos vamos a evaluar?

- Gradient Boosting
- Regresión Logística
- Arboles de decisión
- Random Forests


## Alternativas

### I) Sin quantiles(SimpleImputer).
### II) Con quantiles(SimpleImputer).
### III) Eliminando la columna "tiene_producto_credito_activo".

## División del data set para alternativa I

In [ ]:
# 60% train, 20% test, 20% validation
train_o, not_train_o = train_test_split(DATA_ORIGINAL, test_size=0.4, random_state=42)
validation_o, test_o = train_test_split(not_train_o, test_size=0.5, random_state=42)

train_o.shape, validation_o.shape, test_o.shape

## División del data set para alternativa II y III

In [ ]:
# 60% train, 20% test, 20% validation
train, not_train = train_test_split(DATA_MODIFICADA, test_size=0.4, random_state=42)
validation, test = train_test_split(not_train, test_size=0.5, random_state=42)

train.shape, validation.shape, test.shape

<h3><center>I) Mapper Sin quantiles</center></h3> 

In [ ]:
mapper_o = DataFrameMapper([
    (['sexo'], [LabelBinarizer()]), 
    (['edad'], [StandardScaler()]),
    (['codigo_region'], [OneHotEncoder()]),
    (['ocupacion'], [OneHotEncoder()]),
    (['codigo_canal'], [OneHotEncoder()]),
    (['antiguedad'], [StandardScaler()]), 
    (['tiene_producto_credito_activo'], [SimpleImputer(strategy='most_frequent'), LabelBinarizer()]),
    (['saldo_promedio_cuenta'], [StandardScaler()]),
    (['es_activo'], [LabelBinarizer()]),
], df_out=True) # df_out=True → Es lo que muestra el nombre de la columna

In [ ]:
# train transformado
mapper_o.fit(train_o)

In [ ]:
# train transformado
mapper_o.transform(train_o)

In [ ]:
# Nombres de los faetures
mapper_o.transformed_names_

In [ ]:
np.round(mapper_o.fit_transform(train_o), 2)

<h3><center>II) Mapper con quantiles</center></h3>

In [ ]:
mapper = DataFrameMapper([
    (['sexo'], [LabelBinarizer()]), 
    (['codigo_region'], [OneHotEncoder()]),
    (['ocupacion'], [OneHotEncoder()]),
    (['codigo_canal'], [OneHotEncoder()]),
    (['tiene_producto_credito_activo'], [SimpleImputer(strategy='most_frequent'), LabelBinarizer()]),
    (['es_activo'], [LabelBinarizer()]),
    (['Quantiles_edad'], [StandardScaler()]),
    (['Quantiles_saldos'], [StandardScaler()]),
    (['Quantiles_antiguedad'], [StandardScaler()])
], df_out=True) # df_out=True → Es lo que muestra el nombre de la columna

In [ ]:
# train transformado
mapper.fit(train)

In [ ]:
# train transformado
mapper.transform(train)

In [ ]:
# Nombres de los faetures
mapper.transformed_names_

In [ ]:
np.round(mapper.fit_transform(train), 2)

<h3><center>III) Mapper eliminando la columna de 'tiene_producto_credito_activo'</center></h3>

In [ ]:
mapper_dc = DataFrameMapper([
    (['sexo'], [LabelBinarizer()]), 
    (['codigo_region'], [OneHotEncoder()]),
    (['ocupacion'], [OneHotEncoder()]),
    (['codigo_canal'], [OneHotEncoder()]),
    (['es_activo'], [LabelBinarizer()]),
    (['Quantiles_edad'], [StandardScaler()]),
    (['Quantiles_saldos'], [StandardScaler()]),
    (['Quantiles_antiguedad'], [StandardScaler()])
], df_out=True, default=False) # df_out=True → Es lo que muestra el nombre de la columna

In [ ]:
# Lo entrenamos con train
mapper_dc.fit(train)

In [ ]:
# train transformado
mapper_dc.transform(train)

In [ ]:
# Nombres de los faetures
mapper_dc.transformed_names_

In [ ]:
np.round(mapper_dc.fit_transform(train), 2)

# Evaluación Modelos - Alternativa I - Sin quantiles

#### Programemos una función para evaluar un modelo...

In [ ]:
def evaluate_model_o(model, set_names=('train_o', 'validation_o'), title='', show_cm=True):
    if title:
        display(title)
        
    final_metrics = defaultdict(list)
    
    if show_cm:
        fig, axis = plt.subplots(1, len(set_names), sharey=True, figsize=(15, 3))
    
    for i, set_name in enumerate(set_names):
        assert set_name in ['train_o', 'validation_o', 'test_o']
        set_data = globals()[set_name]  # <- hack feo...

        y = set_data.esta_interesado
        y_pred = model.predict(set_data)
        final_metrics['Accuracy'].append(metrics.accuracy_score(y, y_pred))
        final_metrics['Precision'].append(metrics.precision_score(y, y_pred))
        final_metrics['Recall'].append(metrics.recall_score(y, y_pred))
        final_metrics['F1'].append(metrics.f1_score(y, y_pred))
        
        if show_cm:
            ax = axis[i]
            sns.heatmap(metrics.confusion_matrix(y, y_pred), ax=ax, cmap='Blues', annot=True, fmt='.0f', cbar=False)

            ax.set_title(set_name)
            ax.xaxis.set_ticklabels(['no esta interesado', 'esta interesado'])
            ax.yaxis.set_ticklabels(['no esta interesado', 'esta interesado'])
            ax.set_xlabel('Predicted class')
            ax.set_ylabel('True class')
        
    display(pd.DataFrame(final_metrics, index=set_names))
    if show_cm:
        plt.tight_layout()
        plt.show()

#### Gradient Boosting ####

In [ ]:
gbc_model = Pipeline([
    ('mapper', mapper_o),
    ('classifier', GradientBoostingClassifier(random_state=94)),
])

gbc_model.fit(train_o, train_o.esta_interesado)

evaluate_model_o(gbc_model, title='Gradient Boosting')

#### Regresión Logística ####

In [ ]:
regresion_logistica = LogisticRegression()
lr_model = Pipeline([
    ('mapper', mapper_o),
    ('classifier', LogisticRegression(random_state=14)),
])

lr_model.fit(train_o, train_o.esta_interesado)

evaluate_model_o(lr_model, title='Regresión Logística')

#### Arboles de decisión #### 

In [ ]:
tree_model = DecisionTreeClassifier(random_state=42)

lr_model = Pipeline([
    ('mapper', mapper_o),
    ('classifier', tree_model),
])

lr_model.fit(train_o, train_o.esta_interesado)

evaluate_model_o(lr_model, title='Decision Tree')

#### Random Forests ####

In [ ]:
forest_model = RandomForestClassifier(random_state=42) # n_estimators? max_depth=3?, max_features=2?

rf_model = Pipeline([
    ('mapper', mapper_o),
    ('classifier', forest_model),
])

rf_model.fit(train_o, train_o.esta_interesado)

evaluate_model_o(rf_model, title='Random Forest')

# Evaluación alternativa II - Con quantiles.

#### Programemos una función para evaluar un modelo...

In [ ]:
def evaluate_model(model, set_names=('train', 'validation'), title='', show_cm=True):
    if title:
        display(title)
        
    final_metrics = defaultdict(list)
    
    if show_cm:
        fig, axis = plt.subplots(1, len(set_names), sharey=True, figsize=(15, 3))
    
    for i, set_name in enumerate(set_names):
        assert set_name in ['train', 'validation', 'test']
        set_data = globals()[set_name]  # <- hack feo...

        y = set_data.esta_interesado
        y_pred = model.predict(set_data)
        final_metrics['Accuracy'].append(metrics.accuracy_score(y, y_pred))
        final_metrics['Precision'].append(metrics.precision_score(y, y_pred))
        final_metrics['Recall'].append(metrics.recall_score(y, y_pred))
        final_metrics['F1'].append(metrics.f1_score(y, y_pred))
        
        if show_cm:
            ax = axis[i]
            sns.heatmap(metrics.confusion_matrix(y, y_pred), ax=ax, cmap='Blues', annot=True, fmt='.0f', cbar=False)

            ax.set_title(set_name)
            ax.xaxis.set_ticklabels(['no esta interesado', 'esta interesado'])
            ax.yaxis.set_ticklabels(['no esta interesado', 'esta interesado'])
            ax.set_xlabel('Predicted class')
            ax.set_ylabel('True class')

    display(pd.DataFrame(final_metrics, index=set_names))
    if show_cm:
        plt.tight_layout()
        plt.show()

#### Gradient Boosting ####

In [ ]:
gbc_1_model = Pipeline([
    ('mapper', mapper),
    ('classifier', GradientBoostingClassifier(random_state=94)),
])

gbc_1_model.fit(train, train.esta_interesado)

evaluate_model(gbc_1_model, title='Gradient Boosting')

#### Regresión Logística ####

In [ ]:
lr_1_model = Pipeline([
    ('mapper', mapper),
    ('classifier', LogisticRegression(random_state=14)),
])

lr_1_model.fit(train, train.esta_interesado)

evaluate_model(lr_1_model, title='Regresión Logistica')

#### Arboles de decisión #### 

In [ ]:
tree_model = DecisionTreeClassifier(random_state=42)

ad_1_model = Pipeline([
    ('mapper', mapper),
    ('classifier', tree_model),
])

ad_1_model.fit(train, train.esta_interesado)

evaluate_model(ad_1_model, title='Decision Tree')

#### Random Forests ####

In [ ]:
forest_model = RandomForestClassifier(random_state=42)

rf_1_model = Pipeline([
    ('mapper', mapper),
    ('classifier', forest_model),
])

rf_1_model.fit(train, train.esta_interesado)

evaluate_model(rf_1_model, title='Random Forest')

# Evaluación Modelos - Alternativa III - Eliminando la columna "tiene_producto_credito_activo".

#### Gradient Boosting ####

In [ ]:
gbc_model_ec = Pipeline([
    ('mapperdc', mapper_dc),
    ('classifier', GradientBoostingClassifier(random_state=94)),
])

gbc_model_ec.fit(train, train.esta_interesado)

evaluate_model(gbc_model_ec, title='Gradient Boosting')

#### Regresión Logística ####

In [ ]:
lr_model_ec = Pipeline([
    ('mappercd', mapper_dc),
    ('classifier', LogisticRegression(random_state=94)),
])

lr_model_ec.fit(train, train.esta_interesado)

evaluate_model(lr_model_ec, title='Regresión Logistica')

#### Arboles de decisión #### 

In [ ]:
tree_model = DecisionTreeClassifier(random_state=42)

lr_model = Pipeline([
    ('mapper', mapper_dc),
    ('classifier', tree_model),
])

lr_model.fit(train, train.esta_interesado)

evaluate_model(lr_model, title='Decision Tree')

#### Random Forest #### 

In [ ]:
forest_model = RandomForestClassifier(random_state=42)

rf_model = Pipeline([
    ('mapper', mapper_dc),
    ('classifier', forest_model),
])

rf_model.fit(train, train.esta_interesado)

evaluate_model(rf_model, title='Random Forest')

## Comparativa de las técnicas aplicadas ##

- #### Gradient Boosting¶ ####

| Alternativa | Precision(Train) | Precision(Validation) 
| -- | --- | --- |
| Sin quantiles | 0.714523 | 0.707765 |
| Con quantiles | 0.781776 | 0.780931 |
| Sin columna "tiene_producto_credito_activo" | 0.760914 | 0.762367 |


- #### Regresión Logística  ####

| Alternativa | Precision(Train) | Precision(Validation) 
| -- | --- | --- |
| Sin quantiles | 0.658736 | 0.656354|
| Con quantiles | 0.781370 | 0.786641 |
| Sin columna "tiene_producto_credito_activo" | 0.658736 | 0.656354 |

- #### Arboles de decisión¶ ####

| Alternativa | Precision(Train) | Precision(Validation) 
| -- | --- | --- |
| Sin quantiles | 1.000000 | 0.393015 |
| Con quantiles | 0.951180 | 0.421203 |
| Sin columna "tiene_producto_credito_activo" | 0.927228 | 0.415718 |

- #### Random Forest¶ ####

| Alternativa | Precision(Train) | Precision(Validation) 
| -- | --- | --- |
| Sin quantiles | 1.000000 | 0.55831 |
| Con quantiles | 0.908146 | 0.470312 |
| Sin columna "tiene_producto_credito_activo" | 0.876625 | 0.456061 |

De las técnicas de feature aplicadas, optamos por la de quantiles y además tendremos en cuenta la columna "tiene_producto_credito_activos", ya que los resultados son del %78,09 en validation.

Comparando los resultados obtenidos con cada alternativa, optamos por la que tiene aplicada la división por quantiles. Ademas de elegir esta opción, vamos a mantener la columna "tiene_producto_credito_activos" dentro del dataset.

En la Alternativa II se puede obtener un 78,09% en el conjunto de "validation", lo que nos parece un número aceptable para informar al cliente.

## Valor final de la métrica que podría ser informado al cliente  ##

En este apartado comparamos lo valores obtenidos para la alternativa elegida sobre el conjunto de Test.

In [ ]:
#De la alternativa "con quantiles", evaluaremos los modelos y el de mayor valor de test será el elegido.
evaluate_model(gbc_1_model, title='Gradient Boosting', set_names=('train', 'test', 'validation'), show_cm=False)
evaluate_model(lr_1_model, title='Regresión Logística', set_names=('train', 'test', 'validation'), show_cm=False)
evaluate_model(ad_1_model, title='Arból de decisión', set_names=('train', 'test', 'validation'), show_cm=False)
evaluate_model(rf_1_model, title='Random Forest', set_names=('train', 'test', 'validation'), show_cm=False)

Como conclusión de los resultados, se puede observar que para "Árbol de decisión" y "Random Forest" se esta sobre-entrenando, por lo que procedemos al siguiente paso, ajustar hiperparámetros para obtener mejores resultados.

### Ajustando hiperparámetros del modelo de "Arból de decisión"

Ajustando la profundidad máxima en 5 se puede observar la mejora de los resultados sobre los conjuntos "Validation" y "Test" para la métrica elegida inicialmente.

In [ ]:
tree_model_max = DecisionTreeClassifier(max_depth=5,random_state=42)

ad_2_model = Pipeline([
    ('mapper', mapper),
    ('classifier', tree_model_max),
])

ad_2_model.fit(train, train.esta_interesado)

evaluate_model(ad_2_model, title='Arból de decisión - max_depth=5', set_names=('train', 'test', 'validation'), show_cm=False)

### Ajustando hiperparámetros del modelo de "Random Forest"

Realizamos la misma acción anterior, establecimos una profundidad máxima, esta vez de 6. En este cálculo, se puede observar como mejoro la métrica Re-call.

In [ ]:
forest_model_max = RandomForestClassifier(max_depth=6, random_state=42)

rf_2_model = Pipeline([
    ('mapper', mapper),
    ('classifier', forest_model_max),
])

rf_2_model.fit(train, train.esta_interesado)

evaluate_model(rf_2_model, title='Random Forest - max_depth=5', set_names=('train', 'test', 'validation'), show_cm=False)

### Ajustando hiperparámetros del modelo de "Regresión Logística"

El ajuste realizado fue replicar la clase más pequeña (interesados) hasta que tenga tantas muestras como en la más grande (no interesados), pero de manera implícita.

In [ ]:
lr_2_model = Pipeline([
    ('mapper', mapper),
    ('classifier', LogisticRegression(class_weight="balanced",random_state=14)),
])

lr_2_model.fit(train, train.esta_interesado)

evaluate_model(lr_2_model, title='Regresión Logística - class_weight="balanced"', set_names=('train', 'test', 'validation'), show_cm=False)

Como se puede observar, mejoran Re-call y F1, pero la métrica original elegida empeora mucho.

### Ajustando hiperparámetros del modelo de "Gradient Boosting"

Para este caso establecimos la cantidad máxima de árboles generados en 50. Esto nos permitió mejorar los resultados de la métrica elegida.

In [ ]:
gbc_2_model = Pipeline([
    ('mapper', mapper),
    ('classifier', GradientBoostingClassifier(n_estimators=50, random_state=94)),
])

gbc_2_model.fit(train, train.esta_interesado)

evaluate_model(gbc_2_model, title='Gradient Boosting - n_estimators=50', set_names=('train', 'test', 'validation'), show_cm=False)

# Conclusiones


Primero evaluamos los modelos con los hiperparámetros ajustados.

In [ ]:
evaluate_model(gbc_2_model, title='Gradient Boosting - n_estimators=50', set_names=('train', 'test', 'validation'), show_cm=False)
evaluate_model(lr_2_model, title='Regresión Logística - class_weight="balanced"', set_names=('train', 'test', 'validation'), show_cm=False)
evaluate_model(rf_2_model, title='Random Forest - max_depth=5', set_names=('train', 'test', 'validation'), show_cm=False)
evaluate_model(ad_2_model, title='Arból de decisión - max_depth=5', set_names=('train', 'test', 'validation'), show_cm=False)

A simple vista se puede notar que los modelos no sobreentrenan, esto es por la poca variación entre los resultados de Train y Test.

Siguiendo la elección inicial de la métrica, y basandonos en los resultados del ajuste de hiperparámetros, podemos concluir que la mejor elección para presentar al cliente es la de **Random Forest - Max_depth = 5** con una precisión del 92% de acertar.

La elección se debe a la diferencia que hay sobre los demas modelos en cuanto al porcentaje de acierto.